In [ ]:
import pyprind
import pandas as pd
import os
import sys
import numpy
basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000, stream=sys.stdout)
df = pd.DataFrame()
for s in ('test', 'train'):
   for l in ('pos', 'neg'):
     path = os.path.join(basepath, s, l)
     for file in sorted(os.listdir(path)):
        with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
          txt = infile.read()
        df = df._append([[txt, labels[l]]], ignore_index=True)
        pbar.update()
dfcolumns = ['review', 'sentiment']

In [ ]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [ ]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.rename(columns={'0': 'review', '1': 'sentiment'}, inplace=True)
df.head(3)
df.shape

In [ ]:
df.head(3)

In [ ]:
df.loc[0, 'review'][-50:]

In [ ]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [ ]:
import re
def preprocessor(text):
    text = re.sub('<[^]*', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) +' '.join(emoticons).replace('-', ''))
    return text

In [ ]:
df.loc[0, 'review'][-50:]

In [ ]:
def tokenizer(text):
    return text.split()

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
porter = PorterStemmer()

In [ ]:
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
small_param_grid = [
 {
  'vect__ngram_range': [(1, 1)],
  'vect__stop_words': [None],
  'vect__tokenizer': [tokenizer, tokenizer_porter],
  'clf__penalty': ['l2'],
  'clf__C': [1.0, 10.0]
  },
  {
   'vect__ngram_range': [(1, 1)],
   'vect__stop_words': [stop, None],
   'vect__tokenizer': [tokenizer],
   'vect__use_idf':[False],
   'vect__norm':[None],
   'clf__penalty': ['l2'],
   'clf__C': [1.0, 10.0]
   },
]
lr_tfidf = Pipeline([('vect', tfidf),('clf', LogisticRegression(solver='liblinear'))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid, scoring='accuracy', cv=5, verbose=2, n_jobs=1)
gs_lr_tfidf.fit(X_train, y_train)

In [ ]:
stop = stopwords.words('english')

In [ ]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer



In [ ]:
gs_lr_tfidf.best_score_